# Carga da Dimensão Ativo

Este notebook realiza a carga da dimensão ativo (dim_ativo) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import dense_rank, row_number, monotonically_increasing_id
from pyspark.sql.functions import col, trim, lit, current_timestamp, concat_ws
from pyspark.sql.window import Window

## Start Spark Session

In [ ]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Carga Dimensão Ativo") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")

## Define Delta Table Paths

In [ ]:
# Define caminhos das tabelas Delta
delta_path_stg_cotacao = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica"
delta_path_dim_ativo = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold/dim_ativo"

## Read Source Data

In [ ]:
# Lê a tabela staging de cotação histórica
df_stg = spark.read.format("delta").load(delta_path_stg_cotacao)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_stg_cotacao}")
print(f"Total de registros: {df_stg.count()}")

## Transform Data

In [ ]:
# Gera chave surrogate e seleciona atributos únicos dos ativos
window_spec = Window.orderBy("cd_negociacao")

df_dim_ativo = df_stg.select(
    "cd_negociacao",
    "tp_mercado",
    "nm_empresa",
    "ds_especificacao_papel"
).distinct().select(
    # Gera surrogate key
    row_number().over(window_spec).cast("bigint").alias("sk_ativo"),
    
    # Atributos da dimensão
    col("cd_negociacao").alias("cd_ativo"),
    col("tp_mercado"),
    col("nm_empresa"),
    # Concatena nome da empresa e especificação para descrição completa
    concat_ws(" - ", col("nm_empresa"), col("ds_especificacao_papel")).alias("ds_ativo"),
    # ISIN será preenchido posteriormente quando disponível
    lit(None).cast("string").alias("cd_isin"),
    # Tipo do ativo será derivado do código de negociação posteriormente
    lit("AÇÃO").alias("tp_ativo"),
    
    # Controle temporal
    current_timestamp().alias("ts_insercao"),
    current_timestamp().alias("ts_atualizacao")
)

print("✅ Transformação concluída")
print(f"Total de ativos únicos: {df_dim_ativo.count()}")

# Exibe o schema final
print("\nSchema da dimensão ativo:")
df_dim_ativo.printSchema()

## Write Data

In [ ]:
# Salva a dimensão ativo em formato Delta
df_dim_ativo.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_path_dim_ativo)

print(f"✅ Dimensão ativo salva com sucesso em: {delta_path_dim_ativo}")

## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()